In [238]:
'''
Create variable string. 
'''

def create_variable_string(variable_codes): 
    
    # Initialize empty variable string 
    variable_string = r"NAME,B01001_001E"

    # Iterate through filtered dictionary 
    for code in variable_codes: 
        variable_string = variable_string + ',' + code
    
    return variable_string 

'''
Use the geographic field attributes of the feature classes to create a list of urls 
'''
def construct_api_urls(geo_features, api_key, variable_string): 
    
    api_urls = []
    
    with arcpy.da.SearchCursor(geo_features, ['STATE_FIPS', 'COUNTY_FIPS', 'TRACT_FIPS', 'BLOCKGROUP_FIPS']) as cursor: 
        for row in cursor: 
            state       = row[0]
            county      = row[1] 
            tract       = row[2] 
            block_group = row[3] 

            var_clause    = fr"https://api.census.gov/data/2022/acs/acs5?get={variable_string}"
            geo_clause    = fr"&for=block%20group:{block_group}&in=state:{state}%20county:{county}%20tract:{tract}"
            key_clause    = fr"&key={api_key}"

            api_url       = var_clause + geo_clause + key_clause
            api_urls.append(api_url)
        
    return api_urls

'''
Iterate through API URLs and receive dictionary of responses 
'''

def call_census_api(api_urls):
    
    json_list = [] 
    url_count = 0
    
    # Initialize time 
    time_1  = time.time() 
    
    # Send API call 
    print(len(api_urls))
    print('Continue?')
    
    x = input()
    if x == 'no': 
        quit()  
    else: 
        pass 
    
    for url in api_urls: 
        url_count += 1 
        
        print(f'Call #{url_count} / {len(api_urls)}')
        response = requests.get(url)
        print(">>> Sent request")

        print("Checking status:")
        if response.status_code == 200: 
            json_list.append(response.json())
            print(">>> Good")

        else: 
            print('>>> Error: ', response.status_code)
            
        # Measure and print elapsed time 
        time_2  = time.time() 
        elapsed = round(time_2 - time_1, 2) 
        print(f'Elapsed time:')
        print(f">>> {elapsed}")
        print(" ")
    
    return json_list

In [239]:
'''
Master Function 
'''

def retrieve_acs_json(variable_codes, geo_features, api_key): 
    
    variable_string = create_variable_string(variable_codes)
    api_urls        = construct_api_urls(geo_features, api_key, variable_string)
    json_list       = call_census_api(api_urls)
    
    return json_list

'''
Parameters
'''

variable_codes 
geo_features 
api_key          = r"1749dc1d87964116107a80cd7d76fca300dda59f"

'''
Run
'''
json_list = retrieve_acs_json(variable_codes, geo_output, api_key)

RuntimeError: Cannot find field 'COUNTY_FIPS'